In [1]:
#-----------------------------------
# GLOBAL FEATURE EXTRACTION
#-----------------------------------
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import numpy as np
# import mahotas
import cv2
import os
import h5py

#--------------------
# tunable-parameters
#--------------------
images_per_class = 805
fixed_size       = tuple((500, 500))
train_path       = "dataset/train"
h5_data          = 'output/data.h5'
h5_labels        = 'output/labels.h5'

In [2]:
# Average Blurring 

def Blur(image):
    image=cv2.blur(img,(5,5))
    return image

In [3]:
# Converting to HSV 

def HSV_img(image):
    image=cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
    return image